In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import Library

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# Putusan PN Rantau Prapat

def links():
    base = "https://putusan3.mahkamahagung.go.id/direktori/index/pengadilan/pn-rantau-prapat/kategori/narkotika-dan-psikotropika-1/tahunjenis/putus/tahun/2024/page/"
    urls = []
    page = 1

    while len(urls) < 150:
        resp = requests.get(base + str(page) + ".html")
        soup = BeautifulSoup(resp.text, 'html.parser')

        for entry in soup.select(".spost .entry-c a[href*='/putusan/']"):
            url = entry['href']
            number = entry.get_text().strip()
            urls.append((number, url))

            if len(urls) >= 150:
                break

        page += 1

    return urls[:150]

In [ ]:
# Detail Putusan Pengadilan

def detail(number, url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    court = "PN RANTAU PRAPAT"
    evidence = []
    verdict = ""

    num_elem = soup.find("td", string="Nomor")
    case_number = num_elem.find_next_sibling("td").get_text(strip=True) if num_elem else number

    verdict_section = soup.find("td", string="Catatan Amar")
    if verdict_section:
        verdict_html = verdict_section.find_next_sibling("td")
        full_verdict = verdict_html.get_text(separator=" ", strip=True) if verdict_html else ""

        verdict_text = re.sub(r"(?i)^MENGADILI\s*[:]*\s*", "", full_verdict).strip()

        evidence_match = re.search(r"Menetapkan barang bukti berupa:\s*(.*?)(?= Membebankan kepada|$)", verdict_text, re.IGNORECASE)
        if evidence_match:
            evidence = evidence_match.group(1).split('. ')

        verdict = re.sub(r"Menetapkan barang bukti berupa:.*(?= Membebankan kepada|$)", "", verdict_text, flags=re.IGNORECASE).strip()
    else:
        verdict = ""

    return {
        "No Putusan": case_number,
        "Pengadilan": court,
        "Barang Bukti": "; ".join(evidence).strip(),
        "Amar": verdict
    }

In [ ]:
# Scrapping Putusan

cases = links()

data = []
for number, url in cases:
    data.append(detail(number, url))

df = pd.DataFrame(data)
df.dropna(inplace=True)

df.insert(0, "No", range(1, len(df) + 1))

output_path = '/content/drive/My Drive/PN_Rantau_Prapat_2024.csv'
df.to_csv(output_path, index=False)

In [ ]:
from IPython.display import HTML

print("File saved to:", output_path)
print("Menampilkan 25 data Pertama Putusan PN Rantau Prapat:")

display(HTML(df.head(25).to_html(index=False)))

File saved to: /content/drive/My Drive/PN_Rantau_Prapat_2024.csv
Menampilkan 25 data Pertama Putusan PN Rantau Prapat:


No,No Putusan,Pengadilan,Barang Bukti,Amar
1,932/Pid.Sus/2024/PN Rap,PN RANTAU PRAPAT,"1 (satu) bungkus plastik klip transparan sedang yang berisi narkotika jenis Pil Ekstasi seberat 3,43 (tiga koma empat tiga) gram netto; 1 (satu) unit handphone android merk Oppo warna biru Dimusnahkan; 8.","Menyatakan Terdakwa Nur Afifah Adila Siregar alias Dila tersebut diatas tidak terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana sebagaimana dalam Dakwaan Primair; Membebaskan Terdakwa oleh karena itu dari Dakwaan Primair; Menyatakan Terdakwa Nur Afifah Adila Siregar alias Dila tersebut diatas terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana ?Tanpa Hak Memiliki Narkotika Golongan I Bukan Tanaman?, sebagaimana dalam Dakwaan Subsidair; Menjatuhkan pidana kepada Terdakwa oleh karena itu dengan pidana penjara selama 8 (delapan) tahun dan denda sejumlah Rp1.000.000.000,00 (satu milyar rupiah) dengan ketentuan apabila denda tersebut tidak dibayar diganti dengan pidana penjara selama 3 (tiga) bulan; Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan; Menetapkan Terdakwa tetap ditahan;"
2,943/Pid.Sus/2024/PN Rap,PN RANTAU PRAPAT,"1 (satu) bungkus kertas warna cokelat berisi Narkotika jenis ganja seberat 93, 73 (Sembilan puluh tiga koma tujuh puluh tiga) gram netto; 1 (satu) unit Handphone merk Vivo warna putih; 6;","Menyatakan Terdakwa Rindu Efendy Gultom Alias Pak Joi tersebut diatas terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana ?Tanpa Hak Membeli Narkotika Golongan I Dalam Bentuk Tanaman?, sebagaimana dalam Dakwaan Primair; Menjatuhkan pidana kepada Terdakwa oleh karena itu dengan pidana penjara selama 7 (tujuh) Tahun dan denda sejumlah Rp1.000.000.000,00 (satu milyar rupiah), dengan ketentuan apabila denda tersebut tidak dibayar akan diganti dengan pidana penjara selama 6 (enam) bulan; Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan; Menetapkan Terdakwa tetap ditahan;"
3,946/Pid.Sus/2024/PN Rap,PN RANTAU PRAPAT,,"Menyatakan Terdakwa Joni Pransiscus Sihotang Alias Jon tersebut diatas tidak terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana sebagaimana didakwakan dalam Dakwaan Primair; Membebaskan Terdakwa oleh karena itu dari Dakwaan Primair; Menyatakan Terdakwa Joni Pransiscus Sihotang Alias Jon tersebut diatas terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana ?Tanpa Hak Memiliki Narkotika Golongan I Dalam Bentuk Bukan Tanaman? sebagaimana dalam Dakwaan Subsidair; Menjatuhkan pidana kepada Terdakwa oleh karena itu dengan pidana penjara selama 4 (empat) Tahun dan denda sejumlah Rp.1.000.000.000,00 (satu milyar rupiah) dengan ketentuan apabila denda tersebut tidak dibayar diganti dengan pidana penjara selama 1 (satu) Bulan; Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan; Menetapkan Terdakwa tetap ditahan; Menetapkan barang bukti berupa : 2 (dua) buah plastik klip transparan berisi kristal putih narkotika jenis sabu seberat 0,12 (nol koma satu dua) gram netto; 9 (sembilan) buah plastik klip transparan berisi kristal putih narkotika jenis sabu seberat 0,52 (nol koma lima dua) gram netto;1 (satu) unit sepeda motor merek Honda supra X warna biru hitam dengan nomor polisi BK 4600 VBA; Dikembalikan kepada Terdakwa; Uang tunai senilai Rp. 25.000,- (dua puluh lima ribu rupiah); 1 (satu) unit handphone merek Oppo A31 warna hitam; Dirampas untuk Negara; Membebankan kepada Terdakwa membayar biaya perkara sejumlah Rp5.000,00 (lima ribu rupiah) ;"
4,869/Pid.Sus/2024/PN Rap,PN RANTAU PRAPAT,1 (satu) bungkus plastik makanan warna hijau berisi narkotika jenis daun ganja kering seberat 1 (satu) gram netto; dan 1 (satu) lembar kertas tiktak warna putih; Dimusnahkan;,"MENGADIL I : Menyatakan Terdakwa Bambang Arianto alias Bembeng tersebut di atas, tidak ter